
# Sesión 08 — **Clasificación Multiclase** con Wine Quality (UCI)

En esta versión predecimos la **calidad exacta** (`quality` de 3–9 típicamente) como problema **multiclase**.  
Incluye: carga (red+white), EDA básica, *train/test* estratificado, estandarización, **LogReg**, **Árbol**, **Random Forest**, **SVM**, validación cruzada (macro-F1), **matriz de confusión**, **ROC OvR** y **importancia de variables**.

> Requisitos: `pandas`, `numpy`, `scikit-learn`, `matplotlib`.



## 0) Datos esperados y fuentes UCI
Este notebook espera los archivos:
- `/mnt/data/winequality-red.csv`
- `/mnt/data/winequality-white.csv`

Fuentes (copiar/pegar en navegador si hace falta):
- Red: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
- White: https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv


## 1) Importaciones

In [ ]:

# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,
    f1_score, roc_curve, auc
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

pd.set_option('display.precision', 3)
np.set_printoptions(precision=3, suppress=True)


## 2) Carga y unión (red + white), preparación

In [ ]:

# %%
path_red = Path('data/winequality-red.csv')
path_white = Path('data/winequality-white.csv')
if not path_red.exists() or not path_white.exists():
    raise FileNotFoundError("Descarga los CSV de UCI y colócalos en /mnt/data/.")

red = pd.read_csv(path_red, sep=';')
white = pd.read_csv(path_white, sep=';')
red['type'] = 'red'
white['type'] = 'white'
df = pd.concat([red, white], ignore_index=True)

print("Shape:", df.shape)
df.head()


## 3) Definición de objetivo multiclase y *features*

In [ ]:

# %%
# Objetivo multiclase: calidad exacta
y = df['quality'].astype(int).copy()

# Opcional: filtrar clases muy raras para estabilidad (mantener 3–8 típicamente)
clases_presentes = y.value_counts().sort_index()
print("Distribución de clases (quality):")
print(clases_presentes)

# Features (incluye 'type' como dummies)
X = df.drop(columns=['quality']).copy()
X = pd.get_dummies(X, columns=['type'], drop_first=True)

print("Dimensiones X:", X.shape, "| y:", y.shape)


## 4) EDA rápida

In [ ]:

# %%
X.describe()


In [ ]:

# %%
plt.figure(figsize=(6,4))
plt.hist(y, bins=range(int(y.min()), int(y.max())+2), align='left', rwidth=0.8)
plt.xlabel("quality")
plt.ylabel("Frecuencia")
plt.title("Distribución de 'quality' (multiclase)")
plt.tight_layout()
plt.show()


## 5) Partición *train/test* (estratificada por calidad)

In [ ]:

# %%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
print(X_train.shape, X_test.shape)


## 6) Función de evaluación (macro-F1 + matriz de confusión)

In [ ]:

# %%
def evaluar_modelo_mc(nombre, pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average='macro', zero_division=0)
    print(f"\n=== {nombre} ===")
    print(f"Accuracy: {acc:.3f} | Macro-F1: {f1m:.3f}")
    print("\nReporte de clasificación (por clase):")
    print(classification_report(y_test, y_pred, zero_division=0))
    cm = confusion_matrix(y_test, y_pred)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(include_values=True, xticks_rotation=45)
    plt.title(f"Matriz de confusión — {nombre}")
    plt.tight_layout()
    plt.show()
    return {"name": nombre, "pipe": pipe, "y_pred": y_pred, "acc": acc, "f1_macro": f1m}


## 7) Modelos

In [ ]:

# %%
logreg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, multi_class='multinomial'))
])
res_log = evaluar_modelo_mc("Logistic Regression (multinomial)", logreg, X_train, X_test, y_train, y_test)


In [ ]:

# %%
tree = DecisionTreeClassifier(random_state=42, max_depth=None)
res_tree = evaluar_modelo_mc("Decision Tree", tree, X_train, X_test, y_train, y_test)


In [ ]:

# %%
rf = RandomForestClassifier(random_state=42, n_estimators=500, max_depth=None)
res_rf = evaluar_modelo_mc("Random Forest", rf, X_train, X_test, y_train, y_test)


In [ ]:

# %%
svm_rbf = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", SVC(kernel='rbf', probability=True, random_state=42))
])
res_svm = evaluar_modelo_mc("SVM (RBF)", svm_rbf, X_train, X_test, y_train, y_test)


### 7.1) Validación cruzada (macro-F1, k=5)

In [ ]:

# %%
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for nombre, pipe in [
    ("LogReg", logreg),
    ("DecisionTree", tree),
    ("RandomForest", rf),
    ("SVM-RBF", svm_rbf),
]:
    scores = cross_val_score(pipe, X, y, cv=cv, scoring='f1_macro')
    print(f"{nombre}: Macro-F1 = {scores.mean():.3f} ± {scores.std():.3f}")


## 8) ROC One-vs-Rest (multiclase)

In [ ]:

# %%
def plot_roc_ovr_multiclass(nombre, base_estimator, X_train, X_test, y_train, y_test):
    classes = np.unique(y_train)
    y_train_bin = label_binarize(y_train, classes=classes)
    y_test_bin = label_binarize(y_test, classes=classes)

    clf = OneVsRestClassifier(base_estimator)
    clf.fit(X_train, y_train_bin)

    if hasattr(clf, "predict_proba"):
        y_score = clf.predict_proba(X_test)
    else:
        scores = clf.decision_function(X_test)
        if scores.ndim == 1:
            scores = scores[:, None]
        # Normalización min-max por clase
        minv = scores.min(axis=0, keepdims=True)
        maxv = scores.max(axis=0, keepdims=True)
        y_score = (scores - minv) / (maxv - minv + 1e-12)

    plt.figure(figsize=(7,5))
    for i, c in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"Clase {c} (AUC={roc_auc:.2f})")
    plt.plot([0,1], [0,1], linestyle='--')
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title(f"ROC OvR — {nombre}")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Ejemplos con LogReg y SVM
plot_roc_ovr_multiclass("Logistic Regression (multinomial)",
                        LogisticRegression(max_iter=1000, multi_class='ovr'),
                        X_train, X_test, y_train, y_test)

plot_roc_ovr_multiclass("SVM (RBF)",
                        SVC(kernel='rbf', probability=True, random_state=42),
                        X_train, X_test, y_train, y_test)


## 9) Importancia de variables (árboles/bosques)

In [ ]:

# %%
rf_fit = RandomForestClassifier(random_state=42, n_estimators=500)
rf_fit.fit(X_train, y_train)
importances = rf_fit.feature_importances_
idx = np.argsort(importances)[::-1]

print("Top 12 variables por importancia:")
for i in idx[:12]:
    print(f"{X.columns[i]:30s} {importances[i]:.4f}")

plt.figure(figsize=(8,5))
top_k = 12
plt.bar(range(top_k), importances[idx][:top_k])
plt.xticks(range(top_k), [X.columns[i] for i in idx[:top_k]], rotation=45, ha='right')
plt.ylabel("Importancia (Gini)")
plt.title("Importancia de variables — Random Forest")
plt.tight_layout()
plt.show()



## 10) Notas didácticas
- El problema es **desbalanceado** (pocas muestras en calidades extremas). Reports macro-F1 y matriz de confusión ayudan a interpretar.
- **Escalado** es clave para LogReg y SVM.
- Para estabilizar, puedes **agrupar**: p.ej., 3–4=“baja”, 5–6=“media”, 7–8=“alta” (convierte en 3 clases).
- Considera usar *GroupKFold* si deseas separar por `type` o por lote de producción (si existiera).
